In [1]:
!pip install spark-sklearn

In [2]:
import pandas as pd
!wget http://localhost/mimic_proc_f_test.csv.gz
mimic = pd.read_csv('mimic_proc_f_test.csv.gz')
len(mimic), mimic.columns

In [3]:
columns_data = mimic.columns[1:len(mimic.columns)-4]
data = mimic[columns_data].fillna(0).values
target = mimic['cD#death'].values

In [4]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
digits = load_digits()
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "n_estimators": [10, 20, 40, 80]}
clf = RandomForestClassifier()

In [5]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(clf, param_grid=param_grid)
start = time.time()
gs.fit(digits.data, digits.target)
print(time.time() - start)

In [6]:
from spark_sklearn import GridSearchCV
from spark_sklearn.util import createLocalSparkSession

sc = createLocalSparkSession().sparkContext
gs = GridSearchCV(sc, clf, param_grid=param_grid)

start = time.time()
gs.fit(digits.data, digits.target)
print(time.time() - start)

In [7]:
sc = createLocalSparkSession().sparkContext
gs = GridSearchCV(sc, clf, param_grid=param_grid)

start = time.time()
gs.fit(data, target)
print(time.time() - start)

In [8]:
gs.best_score_, gs.best_params_

In [9]:
# fonte https://databricks.com/blog/2016/02/08/auto-scaling-scikit-learn-with-apache-spark.html